In [5]:
import pandas as pd
import numpy as np

from dataclasses import dataclass
from typing import Any, List, Tuple, Dict
from pandas import DataFrame

In [6]:
@dataclass
class City:
    index : int
    X : float
    Y : float

@dataclass
class Item:
    index : int
    Profit : int
    Weight : int
    Node : int
    
@dataclass
class TTP:
    Name :str = None
    DTYPE : str = None
    Dimension : int = 0
    ITEMS : int = 0
    CAPACITY : int = 0
    MIN_SPEED : float = 0
    MAX_SPEED : float = 0
    RENTING_RATIO : float = 0
    EDGE_W : str = None
    NODE : List[City] = None
    ITEM : List[Item] = None

In [7]:
def read_problem(file_path:str):
    with open(file_path,'r') as file:
        lines = file.readlines()
    
    data = TTP(NODE=[],ITEM=[])
    
    for i , line in enumerate(lines):
        if line.startswith("PROBLEM NAME"):
            data.Name = line.split(':')[-1].strip()
        elif line.startswith("KNAPSACK DATA TYPE"):
            data.DTYPE = line.split(':')[-1].strip()
        elif line.startswith("DIMENSION"):
            data.Dimension = int(line.split(':')[-1].strip())
        elif line.startswith("NUMBER OF ITEMS"):
            data.ITEMS = int(line.split(':')[-1].strip())
        elif line.startswith("DIMENSION"):
            data.Dimension = int(line.split(':')[-1].strip())
        elif line.startswith("MIX SPEED"):
            data.MIN_SPEED = float(line.split(':')[-1].strip())
        elif line.startswith("MAX SPEED"):
            data.MAX_SPEED = float(line.split(':')[-1].strip())
        elif line.startswith("RENTING RATIO"):
            data.RENTING_RATIO = float(line.split(':')[-1].strip())
        elif line.startswith("EDGE_WEIGHT_TYPE"):
            data.EDGE_W = line.split(':')[-1].strip()
        elif line.startswith("NODE_COORD_SECTION"):
            for j in range(1,data.Dimension+1):
                node = lines[i+j].split()
                data.NODE.append(City(index=int(node[0]),X=float(node[1]),Y=float(node[2])))
        elif line.startswith("ITEMS SECTION"):
            for j in range(1,data.ITEMS+1):
                item = lines[i+j].split()
                data.ITEM.append(
                    Item(int(item[0]),int(item[1]),int(item[2]),int(item[3]))
                )
        else:
            pass
    
    return data.NODE , data.ITEM

In [23]:
def generate_ttp_solution(number_of_cities: int, items: List[Item], knapsack_capacity: int) -> Tuple[List[int], List[int]]:
    # Generate a random path (tour)
    path = np.random.permutation(number_of_cities) + 1

    # Initialize knapsack plan with no items picked
    plan = [0] * len(items)
    current_weight = 0

    # Randomly decide to pick up items considering the knapsack capacity
    for i, item in enumerate(items):
        item_weight = item.Weight
        if current_weight + item_weight <= knapsack_capacity:
            decision = np.random.choice([0, 1])
            plan[i] = decision
            current_weight += item_weight * decision

    return path.tolist(), plan

In [9]:
def euclidean_distance(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def calculate_time_and_profit(solution: List[int], plan: List[int], nodes: List[City], items: List[Item], min_speed, max_speed, max_weight):
    total_time = 0
    total_profit = 0
    current_weight = 0

    # Calculate total profit from picked items
    for item, is_picked in zip(items, plan):
        if is_picked:
            total_profit += item.Profit

    # Calculate the total travel time
    for i in range(len(solution)):
        current_city_index = solution[i]
        next_city_index = solution[0] if i == len(solution) - 1 else solution[i + 1]

        current_city = nodes[current_city_index - 1]
        next_city = nodes[next_city_index - 1]

        # Update current weight based on items picked at the current city
        for item, is_picked in zip(items, plan):
            if is_picked and item.Node == current_city_index:
                current_weight += item.Weight

        # Calculate speed based on current weight
        speed = max_speed - (current_weight / max_weight) * (max_speed - min_speed)
        speed = max(speed, min_speed)  # Ensure speed doesn't drop below minimum

        # Distance between current city and next city
        distance = euclidean_distance((current_city.X, current_city.Y), (next_city.X, next_city.Y))

        # Update time with time to next city
        total_time += distance / speed

    return total_time, total_profit

In [10]:
path = './datasets/test-example-n4.txt'
path2 = './datasets/a280-n1395.txt'
node , item = read_problem(file_path=path)
print(node,'\n', item)

[City(index=1, X=0.0, Y=0.0), City(index=2, X=4.0, Y=0.0), City(index=3, X=8.0, Y=3.0), City(index=4, X=0.0, Y=3.0)] 
 [Item(index=2, Profit=34, Weight=30, Node=2), Item(index=3, Profit=40, Weight=40, Node=3), Item(index=4, Profit=25, Weight=21, Node=4)]


In [13]:
max_weight = 80
max_speed = 1
min_speed = 0.1

path = [1,2,3,4]
plan = [0,1,1]
cost = calculate_time_and_profit(path,plan,node,item,min_speed,max_speed,max_weight)
cost

(33.107207533502354, 65)

In [37]:
renting_ratio = 1.516

# @dataclass
# class Genome:
#     path : List[int]
#     plan : List[int]

@dataclass
class Phenome:
    time : float
    profit : float
    net_profit : float = 0

    def __post_init__(self):
        self.net_profit = self.profit - (self.time*renting_ratio)

@dataclass
class Chromosome:
    path : List[int]
    plan : List[int]
    phenome : Phenome = None

    def __post_init__(self):
        # time , profit = calculate_time_and_profit(path,plan,node,item,min_speed,max_speed,max_weight)
        # self.phenome = Phenome(time=time,profit=profit)
        self.phenome = Phenome(*calculate_time_and_profit(path,plan,node,item,min_speed,max_speed,max_weight))

@dataclass
class Population:
    population : List[Chromosome]
    


In [32]:
path,plan = generate_ttp_solution(4,item,80)

In [33]:
g1 = Chromosome(path,plan)
g1

Chromosome(path=[4, 1, 2, 3], plan=[0, 0, 0], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32))

In [35]:
population_size = 10
population = [ Chromosome(*generate_ttp_solution(4,item,80)) for _ in range(population_size)]
population

[Chromosome(path=[2, 4, 1, 3], plan=[0, 0, 1], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[1, 4, 2, 3], plan=[1, 0, 1], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[4, 1, 2, 3], plan=[0, 1, 0], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[1, 4, 3, 2], plan=[0, 0, 0], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[1, 3, 2, 4], plan=[0, 0, 1], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[3, 4, 2, 1], plan=[0, 1, 1], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[4, 3, 1, 2], plan=[1, 0, 1], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[3, 2, 1, 4], plan=[1, 1, 0], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[3, 1, 2, 4], plan=[1, 0, 0], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32)),
 Chromosome(path=[3, 4, 2, 1], plan=[1, 1, 0], phenome=

In [40]:
def tournament_selection(population: List[int], tournament_size: int) -> Chromosome:
    """
    Selects a single Chromosome from the population using tournament selection.

    :param population: An instance of the Population class containing Chromosomes.
    :param tournament_size: The number of Chromosomes to be selected for each tournament.
    :return: The winning Chromosome with the highest net profit.
    """
    # Ensure the tournament size is not larger than the population size
    tournament_size = min(tournament_size, len(population))
    
    # Randomly select 'tournament_size' individuals from the population
    tournament_contestants = np.random.choice(population, size=tournament_size, replace=False)
    
    # Determine the winner based on the highest net profit
    winner = max(tournament_contestants, key=lambda chromo: chromo.phenome.net_profit)
    
    return winner


In [42]:
p1 = tournament_selection(population,2)
p1

Chromosome(path=[2, 4, 1, 3], plan=[0, 0, 1], phenome=Phenome(time=20.0, profit=0, net_profit=-30.32))

In [43]:
def single_swap_mutation(chromosome: Chromosome) -> Chromosome:
    """
    Performs a single swap mutation on the given Chromosome.

    :param chromosome: An instance of the Chromosome class.
    :return: A new Chromosome instance with two path elements swapped.
    """
    # Make a copy of the chromosome's path to avoid mutating the original
    mutated_path = chromosome.path.copy()

    # Select two indices to swap
    idx1, idx2 = np.random.choice(range(len(mutated_path)), size=2, replace=False)

    # Perform the swap
    mutated_path[idx1], mutated_path[idx2] = mutated_path[idx2], mutated_path[idx1]

    # Create a new Chromosome with the mutated path and the same plan
    mutated_chromosome = Chromosome(
        path=mutated_path,
        plan=chromosome.plan.copy(),
        # phenome=chromosome.phenome  # Assuming the phenome does not need to be recalculated for mutation
    )

    return mutated_chromosome

def multiple_swap_mutation(chromosome: Chromosome, number_of_swaps: int) -> Chromosome:
    """
    Performs multiple swap mutations on the given Chromosome.

    :param chromosome: An instance of the Chromosome class.
    :param number_of_swaps: The number of swaps to perform.
    :return: A new Chromosome instance with the specified number of path elements swapped.
    """
    # Make a copy of the chromosome's path to avoid mutating the original
    mutated_path = chromosome.path.copy()

    for _ in range(number_of_swaps):
        # Select two indices to swap, making sure they are different
        idx1, idx2 = np.random.choice(range(len(mutated_path)), size=2, replace=False)

        # Perform the swap
        mutated_path[idx1], mutated_path[idx2] = mutated_path[idx2], mutated_path[idx1]

    # Create a new Chromosome with the mutated path and the same plan
    mutated_chromosome = Chromosome(
        path=mutated_path,
        plan=chromosome.plan.copy()
        # Note: Depending on your implementation, you may need to re-evaluate the phenome after mutation
    )

    return mutated_chromosome
